In [2]:
import pandas as pd
from neuralprophet import NeuralProphet
import pickle
import joblib

/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(


In [11]:
filename = 'np_model2.sav'
main_model = joblib.load(filename)

In [ ]:
with open('/root/weather_app/np_model2.pkl', 'wb') as fout:
    pickle.dump(main_model,fout)  

In [ ]:
with open('/root/weather_app/reg_precip.pkl', 'rb') as fout:
 precip_model = pickle.load(fout)

In [2]:
target = pd.read_csv("data.csv")
target

,ds,y,precip,temp_min
0,1960-01-01,49.0,0.0,30.0
1,1960-01-02,49.0,0.0,29.0
2,1960-01-03,54.0,0.0,35.0
3,1960-01-04,54.0,0.0,36.0
4,1960-01-05,55.0,0.0,33.0
...,...,...,...,...
17457,2023-09-18,73.0,0.0,60.0
17458,2023-09-19,70.0,0.0,60.0
17459,2023-09-20,71.0,0.0,60.0
17460,2023-09-21,72.0,0.0,55.0


In [3]:
periods = 365

In [4]:
target['ds']=pd.to_datetime(target['ds'])

In [6]:
target_precip = target[['ds','precip']]
target_precip.columns = ['ds','y']
target_tmin = target[['ds','temp_min']]
target_tmin.columns = ['ds','y']

In [7]:
tp = NeuralProphet()
tp.set_plotting_backend("matplotlib")
tp.fit(target_precip,freq='D',epochs= 100)

WARNING - (NP.plotting.log_warning_deprecation_plotly) - DeprecationWarning: default plotting_backend will be changed to plotly in a future version. Switch to plotly by calling `m.set_plotting_backend('plotly')`.
WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.971% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 64


Finding best initial lr:   0%|          | 0/256 [00:00<?, ?it/s]

Training: 0it [00:00, ?it/s]

,MAE,RMSE,Loss,RegLoss,epoch
0,0.423636,0.551268,0.728925,0.0,0
1,0.337658,0.450240,0.534391,0.0,1
2,0.230215,0.317394,0.298985,0.0,2
3,0.123760,0.204222,0.115577,0.0,3
4,0.073503,0.171900,0.074564,0.0,4
...,...,...,...,...,...
95,0.063383,0.165051,0.069864,0.0,95
96,0.063609,0.163479,0.069847,0.0,96
97,0.063326,0.165820,0.069844,0.0,97
98,0.063268,0.164769,0.069888,0.0,98


In [8]:
future_precip=tp.make_future_dataframe(target_precip,n_historic_predictions=True,periods=periods)
fcst_precip = tp.predict(future_precip)
fcst_precip[-365:]

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.971% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.972% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.972% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D


Predicting: 273it [00:00, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


,ds,y,yhat1,trend,season_yearly,season_weekly
17462,2023-09-23,NaN,0.002465,0.025880,-0.024551,0.001136
17463,2023-09-24,NaN,0.002440,0.025879,-0.024066,0.000627
17464,2023-09-25,NaN,0.001627,0.025878,-0.023552,-0.000698
17465,2023-09-26,NaN,0.003333,0.025877,-0.023009,0.000465
17466,2023-09-27,NaN,0.002930,0.025876,-0.022439,-0.000507
...,...,...,...,...,...,...
17822,2024-09-17,NaN,-0.000605,0.025517,-0.026588,0.000465
17823,2024-09-18,NaN,-0.001257,0.025516,-0.026267,-0.000507
17824,2024-09-19,NaN,-0.001445,0.025515,-0.025914,-0.001046
17825,2024-09-20,NaN,0.000007,0.025514,-0.025530,0.000022


In [ ]:
#####
#%%timeit
filename = 'reg_precip.sav'
joblib.dump(tp,filename)
#####

In [ ]:
#####
with open('reg_precip.pkl','wb') as filename1:
    pickle.dump(tp, filename1)
#####

In [63]:
tm = NeuralProphet()
tm.set_plotting_backend("matplotlib")
metrics = tm.fit(target_tmin,freq='D',epochs= 100)


WARNING - (NP.plotting.log_warning_deprecation_plotly) - DeprecationWarning: default plotting_backend will be changed to plotly in a future version. Switch to plotly by calling `m.set_plotting_backend('plotly')`.
WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.971% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 64


Finding best initial lr:   0%|          | 0/256 [00:00<?, ?it/s]

Training: 0it [00:00, ?it/s]

In [1]:
future_tmin=tm.make_future_dataframe(target_tmin,n_historic_predictions=True,periods=periods)
fcst_tmin = tm.predict(future_tmin)
fcst_tmin[-365:]

NameError: name 'tm' is not defined

In [ ]:
#####
filename = 'np_reg_tmin.sav'
joblib.dump(tm, filename)
#####

In [ ]:
#####
filename = 'np_reg_precip.sav'
precip_model = joblib.load(filename)
#####

In [48]:
future = fcst_precip[['ds','yhat1']].copy()

In [49]:
future.rename(columns = {'yhat1':'precip'}, inplace = True)

In [50]:
future = future.merge(fcst_tmin[['ds','yhat1']],how='outer',on='ds')

In [51]:
future.rename(columns = {'yhat1':'temp_min'}, inplace = True)

In [46]:
future = future.merge(target[['ds','y']],how='outer',on='ds')

In [39]:
future = future[['ds','y','precip','temp_min']]

In [52]:
future

,ds,precip,temp_min
0,1960-01-01,0.060816,39.840076
1,1960-01-02,0.061225,39.812748
2,1960-01-03,0.060240,39.756355
3,1960-01-04,0.058673,39.796684
4,1960-01-05,0.059053,39.891670
...,...,...,...
17822,2024-09-17,-0.000303,56.942707
17823,2024-09-18,-0.000697,56.921963
17824,2024-09-19,-0.000970,56.863838
17825,2024-09-20,0.000546,56.738834


In [56]:
main_model.restore_trainer()

In [66]:
df_future = main_model.make_future_dataframe(target,n_historic_predictions=True,periods=periods)
forecast = main_model.predict(df_future)
forecast

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.971% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
WARNING - (NP.data.splitting._make_future_dataframe) - Number of forecast steps is defined by n_forecasts. Adjusted to 1.
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.971% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.971% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D


Predicting: 0it [00:00, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


,ds,y,yhat1,lagged_regressor_precip1,lagged_regressor_temp_min1,trend,season_yearly,season_weekly
0,1960-01-01,49.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1960-01-02,49.0,54.678619,-0.710009,8.239253,54.692520,-7.407285,-0.135866
2,1960-01-03,54.0,54.553192,-0.685525,7.955141,54.690174,-7.372572,-0.034028
3,1960-01-04,54.0,56.360649,-0.832424,9.659815,54.687828,-7.332731,0.178164
4,1960-01-05,55.0,56.650627,-0.856907,9.943927,54.685478,-7.287987,0.166115
...,...,...,...,...,...,...,...,...
17458,2023-09-19,70.0,77.345520,-1.444500,16.762619,55.337311,6.523974,0.166115
17459,2023-09-20,71.0,77.258102,-1.444500,16.762619,55.337566,6.518950,0.083464
17460,2023-09-21,72.0,77.065491,-1.444500,16.762619,55.337822,6.508106,-0.098555
17461,2023-09-22,70.0,75.690208,-1.322085,15.342059,55.338074,6.491450,-0.159294
